#**VRBagged-Net**
This notebook implements VRBagged-Net framework, which utilizes the power of deep learning based pre-trained models along with bagging technique, to produce effective results for classification of disaster events. 

In [ ]:
#Libariries 
import os
import numpy
import numpy as np
import scipy as scp
import scipy.misc
import glob
import numpy as np
import csv
import cv2 
import random 

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from sklearn.model_selection import train_test_split

import numpy as np
from tqdm import tqdm
import cv2
import keras

from keras import optimizers
from keras.optimizers import SGD
from keras.optimizers import Adam


In [ ]:
#Authentication from GoogleDrive 
from google.colab import drive 
drive.mount('/content/gdrive') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#**Data Augmentation**
Data augmentation technique is applied to increase the quantity of image in minority class. For this purpose, Augmentor library is utilized to create multiple augmented copies of same image. Number of images in minority class are increased to equalize the quantity of both classes. 


In [ ]:
!pip install Augmentor


In [ ]:
#Code to generate single copy of each image in folder. This cell can be run for 9 time to generate 9 different samples of each image in class Flood related topics
import Augmentor 
p = Augmentor.Pipeline("/content/gdrive/My Drive/MediaEval_2019/Dataset/Train_5181/1") #Path of directory, which stores images of class 1 "Flood related topic"

#Different parameters for augmentation
p.rotate(probability=1, max_left_rotation=20, max_right_rotation=20)
p.flip_left_right(probability=0.5)
p.flip_top_bottom(probability=0.2)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)
p.process()

Executing Pipeline:   0%|          | 0/564 [00:00<?, ? Samples/s]

Initialised with 564 image(s) found.
Output directory set to /content/gdrive/My Drive/MediaEval_2019/Dataset/Train_5181/1/output.

Processing <PIL.Image.Image image mode=RGB size=595x300 at 0x7F6A1A2AD2B0>: 100%|██████████| 564/564 [00:22<00:00, 25.06 Samples/s]


#**Selection of Stratified Random Sample**
Stratified random sample images with same quantity (i-e 3000) is selected from each class. 

In [ ]:
#SELECTION OF RANDOM SAMPLES FROM EACH CLASS 

path_00 ='/content/gdrive/My Drive/MediaEval_2019/Dataset/Train_5181/0'         #Path of directory, which stores images of class 0 or No-Flood related topic
path_01 ='/content/gdrive/My Drive/MediaEval_2019/Dataset/Train_5181/1'         #Path of directory, which stores images of class 1 or Flood related topic

names_00 = os.listdir(path_00)                                                  #os.listdir return number of files in directory of No-Flood related topic
names_01 = os.listdir(path_01)                                                  #os.listdir return number of files in directory of Flood related topic

random_00=random.sample(population=names_00, k=3000)                            #selection of 3000 sample from total population of class No-flood related topic. 
random_01=random.sample(population=names_01, k=3000)                            #selection of 3000 sample from total population of class Flood related topic.

print(random_00)                                                                #represent image id of individual image
os.path.join(path_00, random_00[1])                                             #represent complete path for individual image 

['2803.jpg', '3828.jpg', '1886.jpg', '4817.jpg', '786.jpg', '3952.jpg', '2765.jpg', '2911.jpg', '4726.jpg', '5896.jpg', '3322.jpg', '5133.jpg', '5598.jpg', '1289.jpg', '2371.jpg', '3069.jpg', '1659.jpg', '2376.jpg', '6749.jpg', '6247.jpg', '2227.jpg', '3695.jpg', '3983.jpg', '4109.jpg', '2945.jpg', '4486.jpg', '167.jpg', '3709.jpg', '1919.jpg', '4120.jpg', '5841.jpg', '3705.jpg', '5637.jpg', '3377.jpg', '5347.jpg', '6706.jpg', '6507.jpg', '3964.jpg', '5658.jpg', '4323.jpg', '1293.jpg', '6384.jpg', '5322.jpg', '3961.jpg', '350.jpg', '3609.jpg', '3993.jpg', '2817.jpg', '2074.jpg', '1573.jpg', '3656.jpg', '6230.jpg', '795.jpg', '4727.jpg', '6636.jpg', '1879.jpg', '1381.jpg', '5158.jpg', '3299.jpg', '4151.jpg', '3253.jpg', '2116.jpg', '2181.jpg', '629.jpg', '451.jpg', '3296.jpg', '2528.jpg', '2702.jpg', '1654.jpg', '6197.jpg', '625.jpg', '2278.jpg', '2259.jpg', '5575.jpg', '1916.jpg', '6652.jpg', '450.jpg', '5519.jpg', '50.jpg', '6740.jpg', '1981.jpg', '1505.jpg', '1409.jpg', '5123.jpg', '

'/content/gdrive/My Drive/MediaEval_2019/Dataset/Train_5181/0/3828.jpg'

An Stratified random sample (i-e 3000) is selected from class 0 (No-Flood related Topic).

In [ ]:
#LOAD IMAGES OF CLASS 'NO-FLOOD RELATED TOPIC'

n = 6000                                                                        #n represent total sample size of population
X = np.zeros((n, 224, 224, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

print("Loading images of class 'NoFlood related topic")
print(random_00)                                                              

internal_counter = 0                                                            #Counter for image to be loaded for single class 'No-flood related topic'
external_counter = 0                                                            #Counter for all images to be load from both classes 

for img_path in tqdm(random_00):                                                #iterate to fetch each file from 3000 images for class "No-flood related topic"
    path_randomImages=os.path.join(path_00, random_00[internal_counter])        #append directory of 'No-flood related topic' class images with ids of randomly selected images 
    X[external_counter] = cv2.resize(cv2.imread(path_randomImages), (224, 224)) #The X[external_counter] will load and resize images for class "No-flood related topic"
    y[external_counter] = 0                                                     #y[external_counter] will assign 0 for images of class "No-flood related topic"    
    external_counter+=1                                                         #external counter is increased by one after loading of each image 
    internal_counter+=1                                                         #internal counter is increased by one after loading of each image

print ("Loading of images of class 'No-Flood related topic' is complete")

  0%|          | 0/3000 [00:00<?, ?it/s]

Loading images of class 'NoFlood related topic
['2803.jpg', '3828.jpg', '1886.jpg', '4817.jpg', '786.jpg', '3952.jpg', '2765.jpg', '2911.jpg', '4726.jpg', '5896.jpg', '3322.jpg', '5133.jpg', '5598.jpg', '1289.jpg', '2371.jpg', '3069.jpg', '1659.jpg', '2376.jpg', '6749.jpg', '6247.jpg', '2227.jpg', '3695.jpg', '3983.jpg', '4109.jpg', '2945.jpg', '4486.jpg', '167.jpg', '3709.jpg', '1919.jpg', '4120.jpg', '5841.jpg', '3705.jpg', '5637.jpg', '3377.jpg', '5347.jpg', '6706.jpg', '6507.jpg', '3964.jpg', '5658.jpg', '4323.jpg', '1293.jpg', '6384.jpg', '5322.jpg', '3961.jpg', '350.jpg', '3609.jpg', '3993.jpg', '2817.jpg', '2074.jpg', '1573.jpg', '3656.jpg', '6230.jpg', '795.jpg', '4727.jpg', '6636.jpg', '1879.jpg', '1381.jpg', '5158.jpg', '3299.jpg', '4151.jpg', '3253.jpg', '2116.jpg', '2181.jpg', '629.jpg', '451.jpg', '3296.jpg', '2528.jpg', '2702.jpg', '1654.jpg', '6197.jpg', '625.jpg', '2278.jpg', '2259.jpg', '5575.jpg', '1916.jpg', '6652.jpg', '450.jpg', '5519.jpg', '50.jpg', '6740.jpg', '1

100%|██████████| 3000/3000 [13:10<00:00,  3.79it/s]

Loading of images of class 'No-Flood related topic' is complete


An Stratified random sample (i-e 3000) is selected from class 1 (Flood related Topic).

In [ ]:
#LOAD IMAGES OF CLASS 'FLOOD RELATED TOPIC'

print("Loading images of class 'Flood related topic")
internal_counter = 0                                                            #Counter for image to be loaded for single class 'Flood related topic' 
for img_path in tqdm(random_01):                                                #iterate to fetch each file from 3000 images for class "Flood related topic" 
    path_randomImages=os.path.join(path_01, random_01[internal_counter])        #append directory of 'Flood related topic' class images with ids of randomly selected images 
    X[external_counter] = cv2.resize(cv2.imread(path_randomImages), (224, 224)) #The X[external_counter] will load and resize images for class "Flood related topic"  
    y[external_counter] = 1                                                     #y[external_counter] will assign 1 for images of class "Flood related topic"    
    external_counter+=1                                                         #external counter is increased by one after loading of each image 
    internal_counter+=1                                                         #internal counter is increased by one after loading of each image
print ("Loading of images of class 'Flood related topic' is complete")

  0%|          | 0/3000 [00:00<?, ?it/s]

Loading images of class 'Flood related topic


100%|██████████| 3000/3000 [13:51<00:00,  3.61it/s]

Loading of images of class 'Flood related topic' is complete


**Train and Validation Split**
A quantity of 80% is allocated for training and remaining 20% is allocated for validation. 

In [ ]:
#20% OF SELECTED IMAGES ARE ALLOCATED TO VALIDATION SET AND REMAINING 80% ARE USED FOR TRAINING 
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,  stratify=y)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(4800, 224, 224, 3)
(1200, 224, 224, 3)
(4800, 1)
(1200, 1)


In [ ]:
!pip install update keras_applications

     |████████████████████████████████| 51kB 7.2MB/s 


#**Loading Pre-trained Model**
VGG16 (Pre-trained on ImageNet and Places365) is loaded. 

In [ ]:
#LOADING MODEL 
#ls '/content/gdrive/My Drive/MediaEval_2017/Bagging_vgg16Hybrid/Keras-VGG16-places365-master/' 
import sys
sys.path.append('/content/gdrive/My Drive/MediaEval_2019/Keras-VGG16-places365-master/')                        #Loading already saved model 

from vgg16_hybrid_places_1365 import VGG16_Hybrid_1365                                                          #VGG16 Hybrid is pretrained on Imagenet and places365 datasets
base_model = VGG16_Hybrid_1365(weights='places', include_top=True)

559480832/559473444 [==============================] - 6s 0us/step


#**Parameter Setting and Model compilation**
Parameters are set for the retraining of VGG16 and model is compiled with set parameters.

In [ ]:
#PARAMETER SETTING FOR FINETUNNING OF MODEL
x=base_model.output
x = Dropout(0.2)(base_model.output)
x = BatchNormalization()(x, training=False)
pred = Dense(1, activation='sigmoid')(x)
model=Model(inputs=base_model.input,outputs=pred)

opt = keras.optimizers.Adam(lr=0.00001)                                       
#opt = keras.optimizers.Adam(lr=0.00001)                                        

#Unfreeze last few layers to retrain the model 
for layer in model.layers[:-4]:
    layer.trainable = False
 
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

#**VGGHybrid: Retraining of Model**

In [ ]:

model.fit(X_train, y_train, batch_size=16, epochs=40, validation_data=(X_valid, y_valid), shuffle=True)


Epoch 1/40
300/300 [==============================] - 23s 77ms/step - loss: 0.9763 - accuracy: 0.6094 - val_loss: 0.9724 - val_accuracy: 0.7383
Epoch 2/40
300/300 [==============================] - 23s 77ms/step - loss: 0.9692 - accuracy: 0.7619 - val_loss: 0.9677 - val_accuracy: 0.7867
Epoch 3/40
300/300 [==============================] - 24s 80ms/step - loss: 0.9655 - accuracy: 0.7823 - val_loss: 0.9648 - val_accuracy: 0.8058
Epoch 4/40
300/300 [==============================] - 26s 86ms/step - loss: 0.9620 - accuracy: 0.7933 - val_loss: 0.9623 - val_accuracy: 0.8183
Epoch 5/40
300/300 [==============================] - 26s 88ms/step - loss: 0.9596 - accuracy: 0.8002 - val_loss: 0.9602 - val_accuracy: 0.8275
Epoch 6/40
300/300 [==============================] - 25s 84ms/step - loss: 0.9579 - accuracy: 0.7896 - val_loss: 0.9582 - val_accuracy: 0.8333
Epoch 7/40
300/300 [==============================] - 26s 85ms/step - loss: 0.9553 - accuracy: 0.8102 - val_loss: 0.9564 - val_accuracy:

#**VGGHybrid: Prediction on Test Data**
Trained model is tested on unseen images. The unique id for the image of each tweet is saved in comma separated file, along with its prediction confidence. Moreover, the threshold of 0.5 is used for the prediction of either 1=Flood related top, or 0=No-flood related top class. 


In [ ]:
############LABEL PREDICTION FOR TEST-SET##########################

import os
import numpy
import numpy as np
import scipy as scp
import scipy.misc
import glob
import numpy as np
import csv
import cv2
import pandas as pd

from keras.models import load_model,Model

count = 0
arr_id=[]
for img_name in glob.glob('/content/gdrive/My Drive/MediaEval_2019/Dataset/TestSet_1296/*.*'):          #Directory which stores text-set images 
    image_resized = cv2.resize(cv2.imread(img_name), (224, 224)).astype(np.float32)                     #Loading and resizing images 
    image = image_resized.reshape((1, 224, 224, 3))                                         
    prob=model.predict(image)                                                                           #Getting confidence/probabilitic outcome of model for individual image  
    y_class = 0 if prob<0.5 else 1                                                                      #Threshold of 0.5 for prediction of 0=No-flood related or 1=Flood related class 
    
    #extract img_ids for each test-set image
    base=os.path.basename(img_name)
    os.path.splitext(img_name)
    img_name2 = os.path.splitext(base)[0]
    #counter for test-set 
    count+=1
    print(count)

    #Combine img_id, its respective prediction and probabilitic outcome
    d = {'img_id': [img_name2],'pred':y_class,'prob': str(prob).strip("[]") }
    df_pred = pd.DataFrame(data=d)  

    #Save outcome in csv file    
    print(df_pred)
    df_pred.to_csv('/content/gdrive/My Drive/MediaEval_2019/dir_VRoutcome_1/V_outcome_1.csv', mode='a', header=False, index=False)


1
  img_id  pred        prob
0    168     1  0.51707774
2
  img_id  pred        prob
0   4655     0  0.45009667
3
  img_id  pred        prob
0   3736     1  0.51290524
4
  img_id  pred       prob
0   2145     1  0.5350464
5
  img_id  pred      prob
0   2023     0  0.484593
6
  img_id  pred       prob
0   5233     0  0.4500925
7
  img_id  pred        prob
0   3147     0  0.45062703
8
  img_id  pred       prob
0   4293     1  0.5398032
9
  img_id  pred        prob
0    497     0  0.48189753
10
  img_id  pred       prob
0     96     1  0.5265699
11
  img_id  pred        prob
0    194     0  0.48989332
12
  img_id  pred        prob
0   1641     0  0.46717817
13
  img_id  pred       prob
0   5442     0  0.4500953
14
  img_id  pred       prob
0    318     1  0.5453837
15
  img_id  pred        prob
0   5663     0  0.45197868
16
  img_id  pred        prob
0   4442     0  0.45009443
17
  img_id  pred        prob
0   3329     0  0.46129757
18
  img_id  pred        prob
0   3110     0  0.45162717

#**ResNet50 Mode**

**Loading Pretrained Model**

In [ ]:
base_model_resnet = ResNet50(weights='imagenet',include_top=True,input_shape=(224, 224, 3))

102973440/102967424 [==============================] - 2s 0us/step


**ResNet50: Parameter setting**

In [ ]:
x=base_model_resnet.output
x = Dropout(0.3)(base_model_resnet.output)
x = BatchNormalization()(x, training=False)
pred = Dense(1, activation='sigmoid')(x)
model=Model(inputs=base_model_resnet.input,outputs=pred)

opt = keras.optimizers.Adam(lr=0.00001)

for layer in model.layers[:-10]:
   layer.trainable = False
 
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

#**Training of ResNet50**

In [ ]:
model.fit(X_train, y_train, batch_size=10, epochs=50, validation_data=(X_valid, y_valid), shuffle=True)


Epoch 1/50
480/480 [==============================] - 22s 45ms/step - loss: 0.6894 - accuracy: 0.6571 - val_loss: 0.6855 - val_accuracy: 0.7533
Epoch 2/50
480/480 [==============================] - 22s 45ms/step - loss: 0.6801 - accuracy: 0.7846 - val_loss: 0.6767 - val_accuracy: 0.7933
Epoch 3/50
480/480 [==============================] - 22s 45ms/step - loss: 0.6726 - accuracy: 0.7744 - val_loss: 0.6716 - val_accuracy: 0.8100
Epoch 4/50
480/480 [==============================] - 21s 44ms/step - loss: 0.6682 - accuracy: 0.7694 - val_loss: 0.6679 - val_accuracy: 0.8275
Epoch 5/50
480/480 [==============================] - 21s 44ms/step - loss: 0.6648 - accuracy: 0.7717 - val_loss: 0.6649 - val_accuracy: 0.8375
Epoch 6/50
480/480 [==============================] - 21s 45ms/step - loss: 0.6620 - accuracy: 0.7742 - val_loss: 0.6621 - val_accuracy: 0.8475
Epoch 7/50
480/480 [==============================] - 21s 45ms/step - loss: 0.6588 - accuracy: 0.7844 - val_loss: 0.6596 - val_accuracy:

#**Prediction by ResNet50 based Model**

In [ ]:
############LABEL PREDICTION FOR TEST-SET##########################

import os
import numpy
import numpy as np
import scipy as scp
import scipy.misc
import glob
import numpy as np
import csv
import cv2
import pandas as pd

from keras.models import load_model,Model

count = 0
arr_id=[]
for img_name in glob.glob('/content/gdrive/My Drive/MediaEval_2019/Dataset/TestSet_1296/*.*'):          #Directory which stores text-set images 
    image_resized = cv2.resize(cv2.imread(img_name), (224, 224)).astype(np.float32)                     #Loading and resizing images 
    image = image_resized.reshape((1, 224, 224, 3))                                         
    prob=model.predict(image)                                                                           #Getting confidence/probabilitic outcome of model for individual image  
    #y_class = 1 if prob>0.5 else 0                                                                        
    y_class = 0 if prob<0.5 else 1                                                                      #Threshold of 0.5 for prediction of 0=No-flood related or 1=Flood related class 
    
    #extract img_ids for each test-set image
    base=os.path.basename(img_name)
    os.path.splitext(img_name)
    img_name2 = os.path.splitext(base)[0]

    #counter for test-set 
    count+=1
    print(count)
    #Combine img_id, its respective prediction and probabilitic outcome
    d = {'img_id': [img_name2],'pred':y_class,'prob': str(prob).strip("[]") }

    df_pred = pd.DataFrame(data=d)  

    #Save outcome in csv file    
    print(df_pred)
    df_pred.to_csv('/content/gdrive/My Drive/MediaEval_2019/dir_VRoutcome_1/R_outcome_1.csv', mode='a', header=False, index=False)    
    

1
  img_id  pred        prob
0    168     1  0.58999664
2
  img_id  pred        prob
0   4655     0  0.41463235
3
  img_id  pred       prob
0   3736     0  0.4145696
4
  img_id  pred        prob
0   2145     1  0.54402375
5
  img_id  pred       prob
0   2023     0  0.4147556
6
  img_id  pred       prob
0   5233     0  0.4145692
7
  img_id  pred        prob
0   3147     0  0.41455325
8
  img_id  pred       prob
0   4293     0  0.4539747
9
  img_id  pred        prob
0    497     0  0.41577926
10
  img_id  pred       prob
0     96     1  0.5898245
11
  img_id  pred        prob
0    194     0  0.43159294
12
  img_id  pred        prob
0   1641     0  0.42675504
13
  img_id  pred        prob
0   5442     0  0.41455227
14
  img_id  pred       prob
0    318     1  0.5505349
15
  img_id  pred       prob
0   5663     0  0.4147639
16
  img_id  pred        prob
0   4442     0  0.41455314
17
  img_id  pred        prob
0   3329     0  0.41467503
18
  img_id  pred       prob
0   3110     0  0.4159227